In [6]:
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [10]:
import os
print (os.getcwd())

image = Image.open("../Project/img1.jpg")
histogram = image.histogram()
print(len(histogram))
print(histogram)
type(histogram)
#plt.figure(histogram)
#plt.plot()

/Users/kunalkhatri/Desktop/Semester1/CV/Project
768
[5, 5, 2, 8, 11, 8, 17, 22, 18, 25, 37, 34, 46, 47, 63, 66, 75, 105, 121, 111, 144, 167, 160, 214, 183, 215, 186, 206, 204, 187, 202, 201, 191, 183, 207, 201, 204, 222, 222, 223, 260, 245, 249, 292, 305, 320, 343, 357, 373, 434, 484, 528, 592, 659, 763, 807, 880, 946, 955, 980, 995, 1033, 1166, 1211, 1336, 1327, 1416, 1440, 1522, 1536, 1632, 1624, 1668, 1673, 1727, 1766, 1913, 1850, 1980, 1881, 2015, 1879, 1976, 1927, 2007, 2047, 2072, 2067, 2024, 2048, 2163, 2140, 2237, 2253, 2312, 2296, 2388, 2434, 2310, 2374, 2373, 2338, 2303, 2186, 2125, 2016, 1968, 1829, 1780, 1567, 1589, 1404, 1243, 1196, 1189, 1060, 1100, 1040, 954, 942, 862, 878, 810, 814, 724, 793, 704, 719, 668, 608, 605, 550, 593, 574, 534, 464, 482, 463, 453, 446, 430, 432, 412, 390, 383, 355, 351, 356, 358, 359, 373, 352, 305, 305, 302, 243, 281, 281, 272, 282, 275, 235, 261, 254, 261, 238, 249, 235, 246, 227, 231, 269, 238, 229, 249, 261, 240, 230, 244, 221, 218, 199, 18

list

In [2]:
def get_train_test_file_paths_n_labels():
    """
    Get array train_file_paths, train_labels, test_file_paths and test_labels
    """

    # Data loading and data generators set up
    par_data_dir = 'CUB_200_2011/CUB_200_2011'
    images_data_dir = 'CUB_200_2011/CUB_200_2011/images'
    train_test_split_file = os.path.join(par_data_dir, 'train_test_split.txt')
    images_file = os.path.join(par_data_dir, 'images.txt')
    labels_file = os.path.join(par_data_dir, 'image_class_labels.txt')

    # Read the images_file which stores image-id and image-name mapping
    image_file_id_df = pd.read_csv(images_file, sep=' ', header=None)
    image_file_id_mat = image_file_id_df.as_matrix()
    image_id_name_map = dict(zip(image_file_id_mat[:, 0], image_file_id_mat[:, 1]))

    # Read the train_test_split file which stores image-id and train-test split mapping
    image_id_train_test_split_df = pd.read_csv(train_test_split_file, sep=' ', header=None)
    image_id_train_test_split_mat = image_id_train_test_split_df.as_matrix()
    image_id_train_test_split_map = dict(zip(image_id_train_test_split_mat[:, 0],
                                             image_id_train_test_split_mat[:, 1]))

    # Read the image class labels file
    image_id_label_df = pd.read_csv(labels_file, sep=' ', header=None)
    image_id_label_mat = image_id_label_df.as_matrix()
    image_id_label_map = dict(zip(image_id_label_mat[:, 0], image_id_label_mat[:, 1]))

    # Put together train_files train_labels test_files and test_labels lists
    train_image_ids, test_image_ids = [], []
    train_file_paths, test_file_paths = [], []
    train_labels, test_labels = [], []
    for file_id in image_id_name_map.keys():
        file_name = image_id_name_map[file_id]
        is_train = image_id_train_test_split_map[file_id]
        label = image_id_label_map[file_id] - 1  # To ensure labels start from 0

        if is_train:
            train_image_ids.append(file_id)
            train_file_paths.append(os.path.join(images_data_dir, file_name))
            train_labels.append(label)
        else:
            test_image_ids.append(file_id)
            test_file_paths.append(os.path.join(images_data_dir, file_name))
            test_labels.append(label)

    print ("Length of train files list", len(train_file_paths))
    print ("Length of train labels list", len(train_labels))
    print ("Length of test files list", len(test_file_paths))
    print ("Length of test labels list", len(test_labels))

    return train_image_ids, test_image_ids, train_file_paths, test_file_paths, train_labels, test_labels

In [3]:
train_image_ids, test_image_ids, train_file_paths, test_file_paths, train_labels, test_labels = get_train_test_file_paths_n_labels()

Length of train files list 5994
Length of train labels list 5994
Length of test files list 5794
Length of test labels list 5794


In [8]:
X_train = []
y_train = []
for i, y in zip(train_file_paths, train_labels):
    image = Image.open(i)
    histogram = image.histogram()
    if len(histogram) != 768:
        print("error")
        continue
    X_train.append(histogram)
    y_train.append(y)
print ("Length of X_train", len(X_train))
print ("Legth of y_train", len(y_train))


error
error
error
error
Length of X_train 5990
Legth of y_train 5990


In [21]:
print (len(X_train), len(y_train))
X_train, y_train = shuffle(X_train, y_train)

5990 5990


In [29]:
# define support vector classifier
svm = SVC(kernel='linear', probability=True, random_state=42)

# fit model
svm.fit(X_train, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=42,
    shrinking=True, tol=0.001, verbose=False)

In [13]:
X_test = []
y_test = []
for i, y in zip(test_file_paths, test_labels):
    image = Image.open(i)
    histogram = image.histogram()
    if len(histogram) != 768:
        print("error")
        continue
    X_test.append(histogram)
    y_test.append(y)

print(len(X_test))

error
error
error
error
5790


In [14]:
print (len(X_test), len(y_test))

5790 5790


In [31]:
predicts = svm.predict(X_test)

In [33]:
accuracy_score(predicts, test_labels)

0.02243700379703141

In [22]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

In [23]:
print (y_train[:20])
clf.fit(X_train, y_train)

[113, 193, 55, 148, 84, 120, 168, 194, 131, 130, 129, 58, 51, 32, 79, 131, 68, 3, 102, 115]


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [24]:
accuracy_score(clf.predict(X_test), y_test)

0.0051813471502590676